Note: By default, pyanno currently only works with python 2.7. To import pyanno, also make sure that all the required packages are installed. 

In [ ]:
import pyanno

# Example Annotations

In [7]:
ls

1999-Clinton.txt                        Sites/
APS1950model2                           VirtualBox VMs/
APSabstracts1950s.csv                   anaconda/
Anna Evans - Random Walk project.ipynb  annotation1.txt
Applications/                           bash_profile
Code for James' Class.ipynb             class_web.html
Content Analysis 2 - Words.ipynb        clinton/
Content Analysis 3 - Clusters.ipynb     clinton-docs.gz*
Content Analysis 4 - Spaces.ipynb       lda-example.ipynb
Content Analysis 5 - Reliability.ipynb  lott/
Desktop/                                nltk_data/
Doc2Vec_Toy_Example.ipynb               paj [Nodes].csv
Documents/                              plowData.dat
Downloads/                              pos_tags.png
Dropbox/                                scikit_learn_data/
Google Drive/                           snowplow.dat
Library/                                stopwords.txt
Minimal-Knowledge/                      tasa-20k.hdpmodel.gensim
Movies/                      

In [8]:
from pyanno.annotations import AnnotationsContainer
anno = AnnotationsContainer.from_file("annotation1.txt")

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
anno.annotations

# Bring in your user-generated scores

In [4]:
csvReader = csv.reader(open('APSabstracts1950s.csv', 'r'))
docs = []

In [7]:
for row in csvReader:
    sents = nltk.sent_tokenize(row[3])
    doc = []
    #lowers all cases; removes words in stoplist
    texts = [[word for word in sent.lower().split() if word not in stoplist]for sent in sents]
    for text in texts:
        doc.extend(text)
    docs.append(doc)

### Removes words that appear less than once

In [8]:
from collections import defaultdict
frequency = defaultdict(int)
for doc in docs:
    for token in doc:
        frequency[token] +=1
docs = [[token for token in doc if frequency[token] > 1]for doc in docs]

### Creates bigrams and trigrams

In [10]:
bigram = gensim.models.Phrases(docs)
bigrammed = (bigram[docs])
trigram = gensim.models.Phrases(bigrammed)
trigrammed = (trigram[bigrammed])

# Run word2vec model

In [15]:
start = time.time()
model = gensim.models.Word2Vec(trigrammed, workers=4, batch_words=10000)

for iteration in range(10):
    model.train(trigrammed)

vocab_matrix = model.syn0
vocabulary = model.index2word

model.save('APS1950model2')

end = time.time()
print(end - start)

710.480762959


# Exploring the Model

In [53]:
model = gensim.models.Word2Vec.load('APS1950model2')

In [54]:
model['matter']

array([ 0.11366612,  0.6733855 ,  0.29729691,  0.34947798,  2.24851561,
       -0.57647777,  0.20599896,  0.8607595 , -0.5167051 ,  0.53721344,
       -0.0368433 , -1.206236  ,  0.88823003,  2.4509449 ,  0.60654026,
        0.25592163, -0.51826233,  0.51866895, -0.7907387 , -1.2144618 ,
        1.17498791, -1.45251107, -0.81177002, -0.16462365, -1.34767425,
       -0.28736192,  0.92019171, -1.79114592,  0.72128701,  0.55047262,
       -0.33360377,  0.02686848, -0.61845279,  1.08052635,  0.27933291,
        0.53220147, -0.631818  , -0.7925266 ,  1.39817464, -0.50639665,
       -0.31766775,  1.92057657,  2.07173252,  1.28246355,  2.03123116,
        0.56315064,  0.14823908,  0.70236444, -1.37610137,  0.8148486 ,
        0.94313467, -0.30077243, -0.23178385, -0.23589949,  0.02888884,
        1.83044732, -1.36353433,  0.16784276, -0.60782135,  0.02768349,
       -1.77468729,  1.86033702, -0.38472942, -0.25378212,  0.25974628,
       -0.74237055, -0.11020777, -0.65182447,  2.33068514,  2.85

In [60]:
model.similarity('gas', 'solid')

0.28767101051060878

In [61]:
model.most_similar(positive=['solid', 'liquids'], negative=['gas'], topn=10)

[(u'organic', 0.44844162464141846),
 (u'weak-field', 0.43419504165649414),
 (u'room', 0.42750734090805054),
 (u'solids', 0.41425949335098267),
 (u'moderately', 0.3968007564544678),
 (u'liquid', 0.39475369453430176),
 (u'analyticity', 0.3718644380569458),
 (u'aqueous', 0.3556833863258362),
 (u'degeneracy,', 0.34781524538993835),
 (u'-mesonic_atoms', 0.3470304012298584)]

## Now documents from a folder

In [ ]:
csvReader = csv.reader(open('/clintonAPSabstracts1950s.csv', 'r'))
docs = []
for row in csvReader:
    sents = nltk.sent_tokenize(row[3])
    doc = []
    #lowers all cases; removes words in stoplist
    texts = [[word for word in sent.lower().split() if word not in stoplist]for sent in sents]
    for text in texts:
        doc.extend(text)
    docs.append(doc)
    
    
    
 
from nltk.corpus import PlaintextCorpusReader
corpus_root = '/home/jevans/clinton'
wordlists = PlaintextCorpusReader(corpus_root, '.*')
tokens = wordlist.words('2009-Obama.txt')
print tokens

# Doc2vec example

In [27]:
'''
    Doc2Vec documentation: https://radimrehurek.com/gensim/models/doc2vec.html
    Doc2Vec blog post about installing a fast C compiler to make this soooooo much faster: http://rare-technologies.com/word2vec-in-python-part-two-optimizing/
        - This was a headache for me a bit, so lemme know if theres problems.
        - With these toy examples the speed is not a factor however, so it probably can be disregarded.
'''
patt = r'[,\.-_]+$'

#This is to make sure you have the proper underlying Cython stuff worked up
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

In [28]:
input_docs = ["Hey there, this is a test of the national broadcast system.",
         "Dogs are very cute animals.",
         "Didn't you get a super cute pet the other day?",
         "This is a nightmare, the world is on fire.",
         "How many times do I have to tell you?"]

test_docs = ["My bird makes for a great pet."]

In [32]:
labeled_docs = []
ctr = 0
for doc in input_docs:
    labeled_docs.append(TaggedDocument(words=[re.sub(patt,'', x) for x in doc.lower().split()], tags=['%s'%(str(ctr))])) #Add these TaggedDocuments to a list, which are the datastructure the model takes
    ctr += 1

In [33]:
model = Doc2Vec(labeled_docs, size=100, window=10, min_count=1, workers=4) #Plenty more parameters to mess with. 

for epoch in xrange(0,10):
    random.shuffle(labeled_docs)
    model.train(labeled_docs, total_examples=len(labeled_docs))

### Explore word similarities

In [34]:
print model.similarity('cute', 'pet')
print model.similarity('cute', 'fire')

-0.668450117229
-0.0349147375627


### See which sentences in the input docs are closest to the query

In [36]:
docs_mat = np.zeros((len(model.docvecs), 100))

for ix_ in xrange(len(model.docvecs)):
    vec = model.docvecs[ix_]
    docs_mat[ix_, :] = vec
    
for doc in test_docs:
    vec = model.infer_vector([re.sub(patt, '', x) for x in doc.lower().split()])
    res = cdist(np.reshape(vec, (1,vec.size)), model.docvecs, 'cosine') #distance between doc & every doc in the input_docs
    sorted_ix = np.argsort(res[0])

    sorted_res = res[0, sorted_ix]
    print "query: %s" % doc
    for i in range(3):
        print "#%d) %s" %(i, input_docs[sorted_ix[i]])
        print "\tdistance:%f" %(sorted_res[i])

query: My bird makes for a great pet.
#0) Didn't you get a super cute pet the other day?
	distance:0.548879
#1) How many times do I have to tell you?
	distance:0.548928
#2) This is a nightmare, the world is on fire.
	distance:1.143749


# Run LDA Topic Model

### Read in a corpus. In this example, the file should be a line-delimeted set of documents.

In [40]:
# There are better (in terms of RAM) ways to load the corpus such that is encoded as it's read.
documents = []
fin       = open(wd + "/tasa.docs-line-delimited-20k", "rb")
for line in fin: documents.append(line.rstrip("\n"))
fin.close()

### Pre-processing: utterly important...can take a while (~1 minute on my desktop)

#### 1) load and remove stop-words

In [42]:
stoplist = set()
fin = open(wd + "/stopwords.txt","rb") # A Gerow special
for w in fin: stoplist.add(w.rstrip("\n"))
fin.close()

In [41]:
# Note the lowercasing...
texts = [[word.lower() for word in document.lower().split() if word not in stoplist] for document in documents]

#### 2) Remove words that appear only once:

In [43]:
all_tokens  = sum(texts, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
texts       = [[word for word in text if word not in tokens_once] for text in texts]

## Other important things people tend to do here, in order of importance:
##    1) Deal with punctuation.
##    2) Remove words with a low and high document frequency.
##    3) Remove words below a minimum average TF*IDF value.
##    4) Stem words (NLTK has a couple good stemmers).
##    5) Remove words not found in a dictionary (I have a great / huge dictionary; can take a while).
##    6) Compute and include bigrams that have a collocational strength above some threshold (I use a top n sort of thing).
##       scikit-learn has a great, but convoluted collocation extraction implementation.

## The best rule of thumb I've heard for 2-4 is that the resulting dictionary should be between 15k and 25k words.

### Create the dictionary (also called the vocabulary)

In [44]:
dictionary = corpora.Dictionary(texts)

### Convert our corpus to the gensim sparse representation. 

In [45]:
#This is the great part of the gensim implementations
corpus = [dictionary.doc2bow(text) for text in texts]

In [46]:
#### Don't need to run this for this example####

## Getting to this point may take a while on large corpora. Luckily, you can serialize
##   the corpus in the sparse representation for future use:

corpora.MmCorpus.serialize(wd + '/tasa.mm', corpus) # store to disk, for later use
dictionary.save(wd + '/tasa.dict')

## And read it back in:
corpus     = corpora.MmCorpus(wd + '/tasa.mm')
dictionary = corpora.Dictionary.load(wd + '/tasa.dict')

## And now some topic modeling

In [47]:
## Here, I'm using Hierarchical Dirichlet Processe (HDP)
##   which is just LDA that fits its own parameter for the number of topics. Gensim has
##   a handful of other models, but HDP is one of the more advanced.
## You may get some warnings for the first few iterations here.
hdpmodel = models.HdpModel(corpus, id2word=dictionary)

### Print the topics (a bit ugly, but you get the picture...)

In [48]:
# Notice how harmful punctuation can be.
hdpmodel.print_topics(topics=-1,topn=15) # -1 prints all topics

## These topics, to me, don't seem great -- probably because of the small corpus and negligent pre-processing.

[u'topic 0: 0.003*it + 0.003*is + 0.002*he + 0.002*that + 0.002*was + 0.002*you + 0.002*as + 0.002*be + 0.002*but + 0.001*at + 0.001*on + 0.001*an + 0.001*or + 0.001*could + 0.001*his',
 u'topic 1: 0.003*is + 0.003*that + 0.002*are + 0.002*have + 0.002*you + 0.002*it + 0.001*or + 0.001*i + 0.001*their + 0.001*he + 0.001*but + 0.001*not + 0.001*this + 0.001*was + 0.001*be',
 u'topic 2: 0.003*is + 0.002*that + 0.002*you + 0.001*it + 0.001*are + 0.001*as + 0.001*be + 0.001*from + 0.001*have + 0.001*he + 0.001*they + 0.001*or + 0.001*not + 0.001*people + 0.001*was',
 u'topic 3: 0.002*is + 0.002*that + 0.001*socrates" + 0.001*you + 0.001*it + 0.001*be + 0.001*not + 0.001*are + 0.001*out + 0.001*was + 0.001*she + 0.001*has + 0.001*on + 0.001*less + 0.001*milk',
 u'topic 4: 0.002*is + 0.001*on + 0.001*that + 0.001*you + 0.001*it + 0.001*world, + 0.001*himself + 0.001*can + 0.001*he + 0.001*was + 0.001*are + 0.001*their + 0.001*gertrude + 0.001*cabin. + 0.001*gain',
 u'topic 5: 0.002*is + 0.00

In [49]:
## At this point, there's lots you can do with the topics...
## You can also serialize the fit model for later use:
hdpmodel.save(wd + "/tasa-20k.hdpmodel.gensim")